In [1]:
import netCDF4
from netCDF4 import Dataset
import os
from os.path import dirname
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib 
import matplotlib.pyplot as plt
from math import pi
from numpy import cos,sin
from scipy.spatial import cKDTree
from numpy import absolute as abs
import numpy.ma as ma
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib import ticker
import ipywidgets as widgets

In [2]:
# Choose folder and resolution
folder = dirname(dirname(os.getcwd())) + '\\Data_WQ\\' #The folder where the datasets are
resolution = '110m' # Change border resolution for the maps. Insert value in meters (e.g. '100m')

In [3]:
## Open dataset and store variables in var_values dictionary. Units are in var_units dictionary. Names in var_names
'''
Datasets:
    1: CHL-model
    2: CHL-satellite
    3: DOXYL-model
    4: NITR-model
    5: PHOS-model
    6: SPM-satellite
'''


def select_dataset(num_dataset):
    # Define global variables
    global var_values, var_units, var_names, selected_variable, text, years_list, years_select, years
    
    # Open dataset
    if num_dataset == 2:
        file = folder + "dataset-CHL-satellite-daily.nc"
        selected_variable = 'CHL'
        text = 'Mass concentration of chlorophyll-a in sea water - satellite'
    elif num_dataset == 3:
        file = folder + "dataset-DOXYL-model-daily.nc"
        selected_variable = 'o2'
        text = 'Mole concentration of dissolved molecular oxygen in sea water - model'
    elif num_dataset == 4:
        file = folder + "dataset-NITR-model-daily.nc"
        selected_variable = 'no3'
        text = 'Mole concentration of nitrate in sea water - model'
    elif num_dataset == 5:
        file = folder + "dataset-PHOS-model-daily.nc"
        selected_variable = 'po4'
        text = 'Mole concentration of phosphate in sea water - model'
    elif num_dataset == 6:
        file = folder + "dataset-SPM-satellite-monthly.nc"
        selected_variable = 'SPM'
        text = 'Mass concentration of inorganic suspended matter in sea water - satellite'
    else:
        file = folder + "dataset-CHL-model-daily.nc"
        selected_variable = 'chl'
        text = 'Mass concentration of chlorophyll-a in sea water - model'

    dataset = Dataset(file, "r")


    var_values = {}
    var_units = {}
    var_names = {}

    for k in dataset.variables.keys():
        # Standardize keys 'latitude' and 'longitude'
        if k == 'lat':
            k_new = 'latitude'
        elif k == 'lon':
            k_new = 'longitude'
        else:
            k_new = k

        var_units[k_new] = dataset.variables[k].units # Save units
        var_names[k_new] = dataset.variables[k].long_name # Save name
        if 'time' in var_names[k_new]: #Save time
            var_values[k_new] = netCDF4.num2date(dataset.variables[k][:],var_units[k_new], only_use_cftime_datetimes=False, only_use_python_datetimes = True)
        else:
            var_values[k_new] = dataset.variables[k][:]

        # Remove single-dimensional entries
        var_values[k_new] = np.squeeze(var_values[k_new])

    # Close dataset
    dataset.close()
    print('Dataset loaded. Using dataset {}'.format(file))

In [4]:
def plot_timeseries(lat, lon):
    # Find the nearest latitude and longitude
    lat_idx = np.abs(var_values['latitude'] - lat).argmin()
    lon_idx = np.abs(var_values['longitude'] - lon).argmin()
    print('Closest lat-lon:', var_values['latitude'][lat_idx], var_values['longitude'][lon_idx])
    
    # Get all time records of variable at indices [iy,ix]
    all_records_coordinates = var_values[selected_variable][:,lat_idx,lon_idx]
    if ma.is_masked(var_values[selected_variable][:,lat_idx,lon_idx]):
        print('No data in that coordinate')

    # Plot time series
    plt.figure(figsize=(16,4))
    plt.plot_date(var_values['time'],all_records_coordinates,fmt='-')
    plt.grid()
    plt.ylabel(var_units[selected_variable])
    
    plt.title('%s at Lon=%.2f, Lat=%.2f' % (text, var_values['longitude'][lon_idx], var_values['latitude'][lat_idx]))

In [5]:
print('Choose dataset')
interact_manual(select_dataset, num_dataset=[('Chlorophyll-model', 1), ('Chlorophyll-satellite', 2), ('Oxygen-model', 3), ('Nitrate-model', 4), ('Phosphate-model', 5), ('Suspended matter-satellite', 6)])
print('----------------------------------------------------------------------------------------------')

Choose dataset


interactive(children=(Dropdown(description='num_dataset', options=(('Chlorophyll-model', 1), ('Chlorophyll-sat…

----------------------------------------------------------------------------------------------


In [6]:
print('IMPORTANT: Read the dataset (previous cell) before running this cell for the first time')
print('Plot the timeseries of the dataset in the closest coordinates from the selected coordinates')
lat_input = widgets.FloatSlider(
    value=54.0,
    min=50.0,
    max=56.0,
    step=0.1,
    description='Latitude:',
    continuous_update=False,
    orientation='vertical',
    disabled=False
)


lon_input = widgets.FloatSlider(
    value=3.0,
    min=-2.0,
    max=10.0,
    step=0.1,
    description='Longitude:',
    continuous_update=False,
    disabled=False
)
interact(plot_timeseries, lat = lat_input, lon = lon_input)

IMPORTANT: Read the dataset (previous cell) before running this cell for the first time
Plot the timeseries of the dataset in the closest coordinates from the selected coordinates


interactive(children=(FloatSlider(value=54.0, continuous_update=False, description='Latitude:', max=56.0, min=…

<function __main__.plot_timeseries(lat, lon)>